In [ ]:
import yaml
import datetime
from tqdm.notebook import tqdm, trange
import time
import json
import psycopg2
import netrc
import re
import os


# Globals

In [ ]:
datadir = '.'
jsonfile =  os.path.join(datadir, 'ww_update_data_combined.json')

# Load Data

In [ ]:
print("reusing %s last modified: %s" % (jsonfile, time.ctime(os.path.getmtime(jsonfile))))
with open(jsonfile, 'r') as file:
     update_data = json.load(file)

## List content

In [ ]:
variants = [ k for k in update_data.keys() ]
variants

In [ ]:
cities_list = list({c for v in update_data.values() for c in v.keys() })
cities_list

## Check data

In [ ]:
# check the uploaded curve data for one example
# NOTE this is clipped using only_start_from
#update_data['B.1.1.7']['Zürich (ZH)']["timeseriesSummary"]
update_data['B.1.617.2']['Altenrhein (SG)']['timeseriesSummary']

In [ ]:
# check the (copy-pasted) heatmap data for one example
# NOTE this is clipped using only_start_from
#update_data['B.1.1.7']['Zürich (ZH)']["mutationOccurrences"]
update_data['B.1.617.2']['Altenrhein (SG)']['mutationOccurrences']

## Upload to Cov-Spectrum

In [ ]:
dbhost='id-hdb-psgr-cp61.ethz.ch'

In [ ]:
# load from netrc
dbuser,dbpass=netrc.netrc().authenticators(dbhost)[0::2]

# alternative: input box
#dbuser = input(f"Enter username for database {dbhost}:\n")
#dbpass = input(f"Enter password for user {dbuser}:\n")

# alternative: enviro
#dbuser = os.environ['DB_USERNAME'],
#dbpass = os.environ['DB_PASSWORD'],

dbuser

In [ ]:
dbconn = psycopg2.connect(
    host=dbhost,
    database='sars_cov_2',
    user=dbuser,
    password=dbpass,
    port='5432'
)
dbconn

In [ ]:
cur = dbconn.cursor()
cur

In [ ]:
for pango in variants:
    for city in cities_list:
        cur.execute("""
        DO $$
        BEGIN
         IF EXISTS (SELECT ww.data FROM public.spectrum_waste_water_result AS ww WHERE ww.variant_name=%(var)s AND ww.location=%(city)s) THEN
          UPDATE public.spectrum_waste_water_result AS ww SET data=%(data)s WHERE ww.variant_name=%(var)s AND ww.location=%(city)s;
         ELSE
          INSERT INTO public.spectrum_waste_water_result (variant_name, location, data)
          VALUES(%(var)s, %(city)s, %(data)s);
         END IF;
        END
        $$
        """, {'data': json.dumps(update_data[pango][city]).replace('NaN','null'), 'var': pango, 'city': city})

In [ ]:
## Abort DB update !
dbconn.rollback()

In [ ]:
## Save to DB !
dbconn.commit()

In [ ]:
cur.close()
dbconn.close()